In [1]:
!pip install transformers
!pip install datasets

# Dataset Loading 
The dataset used here is 'IMDB DATASET' which provides binary sentiment dataset \
0->negative\
1->positive

In [3]:
from datasets import load_dataset

imdb = load_dataset('imdb')
imdb
# 0->negative
#1->positive
#unsupervised data has no labels about the sentiment of the text in the dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [3]:
imdb['train']['text'][0]

'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, ev

In [4]:
imdb['train']['label'][0]

0

# Tokenization 

In [15]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

def tokenize(samples):
    return tokenizer(samples['text'], truncation=True)

tokenized_imdb = imdb.map(tokenize, batched=True)
tokenized_imdb.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])


/data/home1/shreeyagarg/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [17]:
tokenized_imdb

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})

# Pytorch DataLoader
DataCollatorWithPadding is a utility provided by Hugging Face's Transformers library that automatically handles the padding of input sequences to the same length within a batch, which is crucial when dealing with variable-length inputs like sentences in NLP tasks.It ensures that all the inputs in a batch are of the same length by padding shorter sequences with a specified token, typically the model's padding token

In [18]:
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors='pt')
train_data = DataLoader(
   tokenized_imdb['train'],
    batch_size=8,
    shuffle=True,
    collate_fn=data_collator
)
val_data=DataLoader(
   tokenized_imdb['test'],
    batch_size=8,
    shuffle=True,
    collate_fn=data_collator
)

In [19]:
for batch in train_data:
    print(batch)
    break

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': tensor([[  101,  2023,  2143,  ...,     0,     0,     0],
        [  101,  1045,  2074,  ...,     0,     0,     0],
        [  101,  1996,  3441,  ...,     0,     0,     0],
        ...,
        [  101,  1996, 10576,  ...,  2518,  2055,   102],
        [  101,  2061,  2043,  ...,     0,     0,     0],
        [  101,  2004,  1037,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([0, 1, 1, 0, 0, 0, 1, 0])}


# Fine Tuning BERT Model

In [6]:

from transformers import AutoModelForSequenceClassification, AdamW
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
optimizer=AdamW(model.parameters(), lr=2e-5)


/data/home1/shreeyagarg/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/data/home1/shreeyagarg/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/data/home1/shreeyagarg/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['cl

## Training

In [7]:
model.train()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [2]:
import torch
from tqdm.notebook import tqdm 

In [12]:
epochs=3
# accu=[]
for epoch in tqdm(range(epochs), total=epochs):  # Single epoch for demonstration
    model.train()
    train_loss=0.0
    train_correct=0
    total_pred=0
    for batch in tqdm(train_data):
        input_ids, attention_mask, labels = batch['input_ids'], batch['attention_mask'], batch['labels']

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        # print(outputs)
        loss = outputs.loss
        logits = outputs.logits
        # print(logits)
        # Compute predictions
        predictions = torch.argmax(logits, dim=-1)
        train_correct += (predictions == labels).sum().item()
        train_loss += loss.detach().cpu().item() / len(train_data)   
        total_pred+=predictions.size(0)
                # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    train_acc=train_correct/total_pred
    print(f"Epoch {epoch + 1} - Training loss: {train_loss / len(train_data)}, Train Accuracy: {train_acc}")
    model.eval()
    val_loss=0.0
    val_acc=0.0
    correct_predictions=0
    total_predictions=0
    with torch.no_grad():
        for batch in val_data:
            batch = {k: v for k, v in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            val_loss += loss.item()
            
            # Get predictions
            predictions = torch.argmax(outputs.logits, dim=-1)
            correct_predictions += (predictions == batch['labels']).sum().item()
            total_predictions += predictions.size(0)
                
    avg_val_loss = val_loss / len(val_data)
    val_acc=correct_predictions/total_predictions
    print(f"Epoch {epoch + 1}/{epochs} Val loss: {avg_val_loss}, Val Accuracy: {val_acc}")

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3125 [00:00<?, ?it/s]

Epoch 1 - Training loss: 7.672978848652864e-05, Train Accuracy: 0.90552
Epoch 1/3 Val loss: 0.16940187140241264, Val Accuracy: 0.93428


  0%|          | 0/3125 [00:00<?, ?it/s]

Epoch 2 - Training loss: 3.936923837666521e-05, Train Accuracy: 0.95716
Epoch 2/3 Val loss: 0.17231278654620052, Val Accuracy: 0.93888


  0%|          | 0/3125 [00:00<?, ?it/s]

Epoch 3 - Training loss: 1.9867714725845993e-05, Train Accuracy: 0.97932
Epoch 3/3 Val loss: 0.20598278416097165, Val Accuracy: 0.93088


In [13]:
torch.save(model, 'Bert.pth')

## Testing

In [9]:
model_BERT=torch.load('/data/home1/shreeyagarg/NLP /Bert.pth')

/tmp/ipykernel_2291473/3193597265.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model_BERT=torch.load('/data/home1/shreeyagarg/NLP /Bert.pth')
/data/home1/shreeyagarg/

# Fine tuning DistilBERT

In [4]:
import torch
from transformers import AutoTokenizer, DistilBertForSequenceClassification, AdamW

tokenizer_distil = AutoTokenizer.from_pretrained('distilbert-base-uncased')
def tokenize(samples):
   return tokenizer_distil(samples['text'], truncation=True)

distil_imdb=imdb.map(tokenize, batched=True)
distil_imdb.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])
model_distil = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", torch_dtype=torch.float16)
optimizer=AdamW(model_distil.parameters(), lr=5e-5)


/data/home1/shreeyagarg/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/data/home1/shreeyagarg/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/data/home1/shreeyagarg/anaconda3/lib/python3.11/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/data/home1/shreeyagarg/anaconda3/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in versio

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.weight', 'classifier.weight', 'pre_classifier.bias', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/data/home1/shreeyagarg/anaconda3/lib/python3.11/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [21]:
distil_imdb

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 50000
    })
})

In [5]:
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer_distil, return_tensors='pt')
train_data = DataLoader(
   distil_imdb['train'],
    batch_size=8,
    shuffle=True,
    collate_fn=data_collator
)
val_data=DataLoader(
   distil_imdb['test'],
    batch_size=8,
    shuffle=True,
    collate_fn=data_collator
)

2024-10-02 17:28:32.662707: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-02 17:28:32.681868: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-02 17:28:32.687488: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-02 17:28:32.702586: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-10-02 17:28:33.934445: W tensorflow/comp

In [23]:
for batch in train_data:
    print(batch)
    break

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'input_ids': tensor([[  101,  2043,  1045,  ...,     0,     0,     0],
        [  101,  1008,  1008,  ...,  1996, 22889,   102],
        [  101,  1996,  2755,  ...,     0,     0,     0],
        ...,
        [  101, 11937,  9515,  ...,     0,     0,     0],
        [  101,  1045,  2387,  ...,     0,     0,     0],
        [  101,  2034,  2125,  ...,     0,     0,     0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]]), 'labels': tensor([1, 1, 0, 0, 0, 0, 0, 0])}


In [6]:
model_distil.train()

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [7]:
epochs=3
# accu=[]
for epoch in tqdm(range(epochs), total=epochs):  # Single epoch for demonstration
    model_distil.train()
    train_loss=0.0
    train_correct=0
    total_pred=0
    for batch in tqdm(train_data):
        input_ids, attention_mask, labels = batch['input_ids'], batch['attention_mask'], batch['labels']

        # Forward pass
        outputs = model_distil(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        # print(outputs)
        loss = outputs.loss
        logits = outputs.logits
        # print(logits)
        # Compute predictions
        predictions = torch.argmax(logits, dim=-1)
        train_correct += (predictions == labels).sum().item()
        train_loss += loss.detach().cpu().item() / len(train_data)   
        total_pred+=predictions.size(0)
                # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    train_acc=train_correct/total_pred
    print(f"Epoch {epoch + 1} - Training loss: {train_loss / len(train_data)}, Train Accuracy: {train_acc}")
    model_distil.eval()
    val_loss=0.0
    val_acc=0.0
    correct_predictions=0
    total_predictions=0
    with torch.no_grad():
        for batch in val_data:
            batch = {k: v for k, v in batch.items()}
            outputs = model_distil(**batch)
            loss = outputs.loss
            val_loss += loss.item()
            
            # Get predictions
            predictions = torch.argmax(outputs.logits, dim=-1)
            correct_predictions += (predictions == batch['labels']).sum().item()
            total_predictions += predictions.size(0)
                
    avg_val_loss = val_loss / len(val_data)
    val_acc=correct_predictions/total_predictions
    print(f"Epoch {epoch + 1}/{epochs} Val loss: {avg_val_loss}, Val Accuracy: {val_acc}")
    

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3125 [00:00<?, ?it/s]

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
